In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

In [ ]:
salary = pd.read_csv("survey_results_public.csv")
salary = salary[["Employment","RemoteWork" ,"EdLevel", "YearsCodePro", "DevType", "OrgSize", "Country","LanguageHaveWorkedWith", "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith","ConvertedCompYearly"]]
print(len(salary))
print(salary.dtypes)

In [ ]:
# cleaning operation 2: drop all rows with missing data ("NA")
salary = salary[pd.notna(salary["Employment"])]
salary = salary[pd.notna(salary["RemoteWork"])]
salary = salary[pd.notna(salary["EdLevel"])]
salary = salary[pd.notna(salary["YearsCodePro"])]
salary = salary[pd.notna(salary["DevType"])]
salary = salary[pd.notna(salary["OrgSize"])]
salary = salary[pd.notna(salary["Country"])]
salary = salary[pd.notna(salary["LanguageHaveWorkedWith"])]
salary = salary[pd.notna(salary["DatabaseHaveWorkedWith"])]
salary = salary[pd.notna(salary["PlatformHaveWorkedWith"])]
salary = salary[pd.notna(salary["ConvertedCompYearly"])]
print(len(salary))

In [ ]:
# cleaning operation 3: focus on data with full-time employment
salary = salary[salary["Employment"] == "Employed, full-time"]
print(len(salary))

In [ ]:
# cleaning operation 4: focus on data in USA
salary = salary[salary["Country"] == "United States of America"]
print(len(salary))

In [ ]:
# cleaning operation 5: focus on data for developers
salary = salary[salary["DevType"].str.contains("Developer")]
print(len(salary))

In [ ]:
# cleaning operation 6: drop invalid data
salary = salary[salary["ConvertedCompYearly"] > 50000]
print(len(salary))

In [ ]:
# cleaning operation 7: make "YearsCodePro" contain only numeric values
salary = salary.replace("Less than 1 year",0)
print(len(salary))

In [ ]:
# cleaning operation 8: drop duplicated row 
salary = salary.drop_duplicates()
print(len(salary))

In [ ]:
# cleaning operation 9: rearrange the column
salary = salary[["Country", "EdLevel", "YearsCodePro","Employment","RemoteWork", "OrgSize", "DevType", "LanguageHaveWorkedWith", "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith","ConvertedCompYearly"]]
print(len(salary))

In [ ]:
# cleaning operation 10: use one hot encoding for frone-end, back-end and full-stack

salary["frontEnd"] = 0
salary["frontEnd"].mask((salary["DevType"].str.contains("Developer, full-stack")) | (salary["DevType"].str.contains("Developer, front-end")), 1, inplace=True)

salary["backEnd"] = 0
salary["backEnd"].mask((salary["DevType"].str.contains("Developer, full-stack")) | (salary["DevType"].str.contains("Developer, back-end")), 1, inplace=True)

salary["FrontEnd"] = 0
salary["FrontEnd"].mask((salary["frontEnd"] == 1) & (salary["backEnd"] == 0), 1, inplace=True)

salary["BackEnd"] = 0
salary["BackEnd"].mask((salary["frontEnd"] == 0) & (salary["backEnd"] == 1), 1, inplace=True)

salary["FullStack"] = 0
salary["FullStack"].mask((salary["frontEnd"] == 1) & (salary["backEnd"] == 1), 1, inplace=True)

salary.drop(columns=["frontEnd", "backEnd"], inplace=True)

salary = salary[["Country", "EdLevel", "YearsCodePro","Employment","RemoteWork", "OrgSize", "DevType", "FrontEnd", "BackEnd", "FullStack", "LanguageHaveWorkedWith", "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith","ConvertedCompYearly"]]
salary.drop(columns=["DevType"], inplace=True)  

In [ ]:
# cleaning operation 11: split “LanguageHaveWorkedWith”, create columns to keep track of the number of front-end languages, back-end languages, and system programming languages each person knows
salary["Front-endLan"] = 0
salary["Back-endLan"] = 0
salary["SysProLan"] = 0
salary["LanguageHaveWorkedWith"] = salary["LanguageHaveWorkedWith"].str.split(pat=";")
Front_end =["JavaScript","HTML/CSS", "Swift","TypeScript","Kotlin","Objective-C","Dart"]
Back_end =["Ruby", "SQL", "Go", "Java", "Python", "Elixir", "Perl", "Groovy", "Scala", "Rust", "C#", "Erlang", "PHP", "Clojure"]
System =["Assembly","C++","C","Rust","Go"]

for i, person in salary.iterrows():
    languages = person["LanguageHaveWorkedWith"]
    for language in languages:
        if language in Front_end:
            salary.at[i, "Front-endLan"] = salary.at[i, "Front-endLan"] +1
        elif language in Back_end:
            salary.at[i, "Back-endLan"] = salary.at[i, "Back-endLan"]+1
        elif language in System:
            salary.at[i, "SysProLan"] = salary.at[i, "SysProLan"]+1
            
salary.drop(columns=["LanguageHaveWorkedWith"], inplace=True)  